In [1]:
# Import libraries

import pandas as pd
from datetime import datetime, timedelta
import dateutil.parser as parser
import requests
from pandasql import sqldf


In [2]:
# Read and clean CSV Sources

# Properties:
properties_cols = ['property_id','latitude','longitude','localidad', 'city', 'country']
convert_pro = lambda x: (x.replace('\xad','').title())  # function used to clean the locality column, Chí­A -> Chí­a
df_properties = pd.read_csv('sources\properties.csv', usecols = properties_cols, converters = {'localidad':convert_pro})

# Users:
users_cols = ['property_id','user_id']
df_users = pd.read_csv('sources\\users.csv', usecols = users_cols)

# Visits:
visits_cols = ['scheduled_id','property_id','begin_date','status']  # only the begin_date is used to query the api
convert_vis = lambda x: (parser.parse(x) + timedelta(hours=2))  # function used to change the time UTC-3 to UTC-5 (Colombian time) in begin_date column
df_visits = pd.read_csv('sources\\visits.csv', usecols = visits_cols, converters = {'begin_date': convert_vis})
df_visits['visit_date'] = df_visits['begin_date'].apply(lambda x: datetime.strftime(x, "%Y-%m-%d"))
df_visits = df_visits.drop(columns='begin_date')


In [3]:
# Join CSV sources to create df_base and extend the visit's information

df_base = pd.merge(df_properties,df_visits, on='property_id', how='left')
df_base = pd.merge(df_base,df_users, on='property_id', how='left')
df_base['key_api'] = df_base['latitude'].map(str)+','+df_base['longitude'].map(str)+'/'+df_base['visit_date'].map(str)  # create a new column with a key to api process


In [4]:
# Get data from API

def rainy_day(conditions):
    rain_conditions = ['type_21','type_5','type_6','type_9','type_10','type_11','type_13',\
                       'type_14','type_22','type_23','type_24','type_25','type_26','type_32']
    conditions_list = [i.strip() for i in conditions.split(',')]
    
    for i in conditions_list:
        if i in rain_conditions:
            return True
    
    return False


def request(key):
    API_KEY = '2DAAC28RT2V8Q2SWYQVXV3DZF'
    endpoint = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{key}'
    query_params = {'unitGroup':'metric', 'key':API_KEY, 'include':'days','lang':'id'}
    
    response = requests.get(endpoint, params=query_params).json()

    temp = response['days'][0]['temp']
    weather = rainy_day(response['days'][0]['conditions'])

    return [temp, weather]


df_base_dd = df_base['key_api'].drop_duplicates().to_frame()  # Remove duplicates to reduce API requests and records cost. Visits with the same property and date have the same temperature and weather

# API request:
df_base_dd['api_response'] = df_base_dd['key_api'].apply(lambda key: request(key))  

# Get average temperature and info regarding rainy days (boolean, True = Rainy day):
df_base_dd['temperature'] = df_base_dd['api_response'].apply(lambda x: x[0])
df_base_dd['rainy_day'] = df_base_dd['api_response'].apply(lambda x: x[1])
df_base_dd_data = df_base_dd.drop(columns='api_response')


In [5]:
# Create df_master with information to response challenge questions 

df_master = pd.merge(df_base,df_base_dd_data, on='key_api', how='left')
df_master = df_master.drop(columns=['latitude','longitude','key_api'])

## Challenge questions:

In [6]:

print('1. ¿Cuántas visitas se realizaron en total?')
q1 = len(df_master.query('status == "Done"'))
q1_all = len(df_master)
print(f'    Rta: Se realizaron {q1} visitas sobre un total de {q1_all} visitas programadas, es decir, se llevo a cabo el {round(q1*100/q1_all, 2)}% de las visitas programadas. \n')

print('2. ¿Cuál es el promedio de propiedades por usuario?')
q2 = (df_users.groupby('user_id')['property_id'].count()).mean()
q2_func = lambda x: "propiedades" if int(x)>1 else "propiedad"
print(f'    Rta: Cada usuario tiene en promedio {int(q2)} {q2_func(q2)}.\n')

print('3. ¿Cuál era la temperatura promedio de todas las visitas que realizó el usuario con ID 2?')
q3 = df_master.query('user_id == 2 and status == "Done"')['temperature'].mean()
print(f'    Rta: La temperatura promedio de las visitas realizadas en la propiedad del usuario con ID 2 fue de {round(q3,2)}°C.\n')

print('4. ¿Cuál es la temperatura promedio de las visitas para los días con lluvia?')
q4 = df_master.query('status == "Done" and rainy_day == True')['temperature'].mean()
print(f'    Rta: La temperatura promedio de las visitas realizadas en días con lluvia fue de {round(q4,2)}°C.\n')

print('5. ¿Cuál es la temperatura promedio para las visitas realizadas en la localidad de Suba?')
q5 = df_master.query('localidad == "Suba" and status == "Done"')['temperature'].mean()
print(f'    Rta: La temperatura promedio de las visitas realizadas en la localidad de Suba es de {round(q5,2)}°C.\n')


1. ¿Cuántas visitas se realizaron en total?
    Rta: Se realizaron 139 visitas sobre un total de 425 visitas programadas, es decir, se llevo a cabo el 32.71% de las visitas programadas. 

2. ¿Cuál es el promedio de propiedades por usuario?
    Rta: Cada usuario tiene en promedio 1 propiedad.

3. ¿Cuál era la temperatura promedio de todas las visitas que realizó el usuario con ID 2?
    Rta: La temperatura promedio de las visitas realizadas en la propiedad del usuario con ID 2 fue de 14.6°C.

4. ¿Cuál es la temperatura promedio de las visitas para los días con lluvia?
    Rta: La temperatura promedio de las visitas realizadas en días con lluvia fue de 14.2°C.

5. ¿Cuál es la temperatura promedio para las visitas realizadas en la localidad de Suba?
    Rta: La temperatura promedio de las visitas realizadas en la localidad de Suba es de 14.12°C.



## Additional analysis using SQL:

In [7]:

pysqldf = lambda q: sqldf(q, globals())

query = """ 
        SELECT status AS VISITS_STATUS
        ,COUNT(*) AS N_VISITS 
        ,ROUND( COUNT(*) * 100.0 / SUM(COUNT(*)) OVER() , 2) AS PERCENTAGE_VISITS
        ,ROUND( AVG(temperature) , 2) AS TEMP_AVG
        ,COUNT(*) FILTER (WHERE rainy_day) AS RAINY_DAYS
        ,ROUND( (COUNT(*) FILTER (WHERE rainy_day)) * 100.0 / SUM(COUNT(*)) OVER(PARTITION BY status) , 2) AS RAINY_DAY_PERCENTAGE
        FROM df_master 
        GROUP BY status
        ORDER BY status DESC
        """

pysqldf(query)


,VISITS_STATUS,N_VISITS,PERCENTAGE_VISITS,TEMP_AVG,RAINY_DAYS,RAINY_DAY_PERCENTAGE
0,Done,139,32.71,14.20,126,90.65
1,Cancelled,286,67.29,13.98,256,89.51
